# Spark DataFrames Project Exercise 

Let's get some quick practice with your new Spark DataFrame skills, you will be asked some basic questions about some stock market data, in this case Walmart Stock from the years 2012-2017. This exercise will just ask a bunch of questions, unlike the future machine learning exercises, which will be a little looser and be in the form of "Consulting Projects", but more on that later!

For now, just answer the questions and complete the tasks below.

#### Use the walmart_stock.csv file to Answer and complete the  tasks below!

#### Start a simple Spark Session

In [0]:
from pyspark.sql import SparkSession; 
spark = SparkSession.builder.appName("WalmartStockAnalysis").getOrCreate()


#### Load the Walmart Stock CSV File, have Spark infer the data types.

In [0]:
walmart_stock_df = spark.read.csv("/FileStore/tables/Walmart_stock.csv", header=True, inferSchema=True)


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2526541766972235>:1
----> 1 walmart_stock_df = spark.read.csv("/FileStore/tables/Walmart_stock.csv", header=True, inferSchema=True)

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:729, in DataFrameReader.csv(self, path, schema, sep, encoding, quote, escape, comment, header, inferSchema, ignoreLeadingWhiteSpace, ignoreTrailingWhiteSpace, nullValue, nanValue, positiveInf, negativeInf, dateFormat, timestampFormat, maxColumns, maxCharsPerColumn, maxMal

#### What are the column names?

In [0]:
walmart_stock_df.columns


#### What does the Schema look like?

In [0]:
walmart_stock_df.printSchema()


#### Print out the first 5 columns.

In [0]:
walmart_stock_df.show(5)


#### Use describe() to learn about the DataFrame.

In [0]:
walmart_stock_df.describe().show()


## Bonus Question!
#### There are too many decimal places for mean and stddev in the describe() dataframe. Format the numbers to just show up to two decimal places. Pay careful attention to the datatypes that .describe() returns, we didn't cover how to do this exact formatting, but we covered something very similar. [Check this link for a hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.cast)

If you get stuck on this, don't worry, just view the solutions.

In [0]:
from pyspark.sql.functions import format_number; 
walmart_stock_df.describe().select("summary", *[format_number(walmart_stock_df.describe().select(c).alias(c).first()[c], 2).alias(c) 
for c in walmart_stock_df.columns 
if c != "Date"]).show()


#### Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day.

In [0]:
hv_df = walmart_stock_df.withColumn("HV Ratio", walmart_stock_df["High"] / walmart_stock_df["Volume"])
hv_df.select("Date", "High", "Volume", "HV Ratio").show(5)


#### What day had the Peak High in Price?

In [0]:
walmart_stock_df.orderBy(walmart_stock_df["High"].desc()).select("Date", "High").show(1)


#### What is the mean of the Close column?

In [0]:
from pyspark.sql.functions import mean

walmart_stock_df.select(mean("Close")).show()


#### What is the max and min of the Volume column?

In [0]:
from pyspark.sql.functions import max

walmart_stock_df.select(max("Volume")).show()


In [0]:
from pyspark.sql.functions import min

walmart_stock_df.select(min("Volume")).show()


#### How many days was the Close lower than 60 dollars?

In [0]:
walmart_stock_df.filter(walmart_stock_df["Close"] < 60).count()


#### What percentage of the time was the High greater than 80 dollars ?
#### In other words, (Number of Days High>80)/(Total Days in the dataset)

In [0]:
# Total number of days
total_days = walmart_stock_df.count()

# Number of days High > 80
high_over_80 = walmart_stock_df.filter(walmart_stock_df["High"] > 80).count()

# Percentage
percentage = (high_over_80 / total_days) * 100
print(f"Percentage of days High > 80: {percentage:.2f}%")


#### What is the Pearson correlation between High and Volume?
#### [Hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameStatFunctions.corr)

In [0]:
correlation = walmart_stock_df.corr('High', 'Volume')
print(f"Pearson correlation between High and Volume: {correlation}")


#### What is the max High per year?

In [0]:
from pyspark.sql.functions import year, month, max, avg

# For max High per year
walmart_stock_df.select(year("Date").alias("Year"), "High").groupBy("Year").agg(max("High").alias("Max_High")).orderBy("Year").show()


#### What is the average Close for each Calendar Month?
#### In other words, across all the years, what is the average Close price for Jan,Feb, Mar, etc... Your result will have a value for each of these months. 

In [0]:
from pyspark.sql.functions import year, month, max, avg
# For average Close per calendar month
walmart_stock_df.select(month("Date").alias("Month"), "Close").groupBy("Month").agg(avg("Close").alias("Avg_Close")).orderBy("Month").show()


# Great Job!